this note book will contain a rag system to answer questions about ITCOMP company data



#

## Installing the required libraries

In [1]:
!pip install -q langchain
!pip install -q llama_index
!pip install -q llama-index-llms-llama-cpp
!pip install -q chromadb
!pip install -q sentence-transformers
!pip install -q FAISS

ERROR: Could not find a version that satisfies the requirement FAISS (from versions: none)
ERROR: No matching distribution found for FAISS


In [2]:
#from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import DirectoryLoader , CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

import pandas as pd


from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

## Loading the model

In [3]:
from langchain_community.llms import LlamaCpp

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="..\models\mistral-7b-instruct-v0.2.Q4_K_M.gguf",
    temperature=0.75,
    max_tokens=2000,
    top_p=1,
    verbose=True,  # Verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from ..\models\mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:        

## loading the embedding model

In [4]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_Name = "sentence-transformers/all-MiniLM-L6-v2"
embed_model_Path2 = "..\models\sentence-transformers/all-MiniLM-L6-v2"

# load the embedding model locally
embed_model = HuggingFaceEmbeddings(model_name=embed_model_Path2)

c:\Users\lemon\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\lemon\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\lemon\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\lemon\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is 

## Data loading and treatment

In [5]:
directory = "../Data/mini_data.csv"

def load_docs(directory):
    #loader = DirectoryLoader(directory)
    loader = CSVLoader(directory)
    documents = loader.load()
    return documents

documents = load_docs(directory)

In [6]:
def split_docs(documents, chunk_size=200, chunk_overlap=50 ):
    text_splitter = RecursiveCharacterTextSplitter(
        #separators=[","],
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    docs = text_splitter.split_documents(documents)
    return docs

docs = split_docs(documents)

## Storing the data in a vector store

In [7]:
# Create the vector store 
vector = FAISS.from_documents(documents, embed_model)

In [8]:
# Define a retriever interface
retriever = vector.as_retriever()

## create a template for the questions

In [9]:
# Define prompt template
prompt = ChatPromptTemplate.from_template("""You are a AI assistant for a company called 'IT Comp'. 
                                            your job is to answer questions you get using the context provided.
                                          the answers should be proffesional and informative.
                                          if you don't know the answer you should say so.

<context>
{context}
</context>

Question: {input}""")

## Create a chain to answer the questions

In [10]:
# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Inferencing the model

In [11]:
def get_answer(query):

    answer = retrieval_chain.invoke({"input": query})
    answer = answer["answer"]

    return answer

In [12]:

print("Private Q&A chatbot")
#prompt = "What is the expiration date of the Distinguish credit card with ID 7 and what was the last modification made to it?"
    


if prompt:
    answer = get_answer(prompt)
    print(f"Answer: {answer}")

Private Q&A chatbot


KeyError: "Input to ChatPromptTemplate is missing variables {'context'}.  Expected: ['context', 'input'] Received: ['input']"

# Testing the model

#load the test data
test_data = pd.read_csv("../Data/GPT_test.csv")
test_data

#print the questions that are wronglly answered

for i in range(len(test_data)):
    if test_data["Evaluation"][i] == "Incorrect":
        print("question number: ", i+1)
        print("Question: ", test_data["Question"][i])
        print("Correct Answer: ", test_data["Actual Answer"][i])
        print("Predicted Answer: ", test_data["Model Answer"][i])
        print("\n")
        print("--------------------------------------------------------------------------------------------")

# print the correctlly answered questions
for i in range(len(test_data)):
    if test_data["Evaluation"][i] == "Correct":
        print("question number: ", i+1)
        print("Question: ", test_data["Question"][i])
        print("Correct Answer: ", test_data["Actual Answer"][i])
        print("Predicted Answer: ", test_data["Model Answer"][i])
        print("\n")
        print("--------------------------------------------------------------------------------------------")


# calculate the number of easy and hard questions
easy = 0
hard = 0
for i in range(len(test_data)):
    if test_data['Question Type'][i] == 'Easy':
        easy += 1
    elif test_data['Question Type'][i] == 'Hard':
        hard += 1

print("Number of easy questions: ", easy)
print("Number of hard questions: ", hard)

# calculate the accuracy of the model
easy_correct = 0
hard_correct = 0
# 1 - for easy questions

for i in range(len(test_data)):
    
    if test_data['Question Type'][i] == 'Easy':
        
        if test_data['Evaluation'][i] == "Correct":
            easy_correct += 1

    elif test_data['Question Type'][i] == 'Hard':
        
        if test_data['Evaluation'][i] == "Correct":
            hard_correct += 1

print("Number of easy questions answered correctly: ", easy_correct , "out of ", easy)
print("Number of hard questions answered correctly: ", hard_correct , "out of ", hard)

print("-----------------------------------------------------------------------")
easy_accuracy = easy_correct / easy
hard_accuracy = hard_correct / hard

print("Easy accuracy: ", easy_accuracy)
print("Hard accuracy: ", hard_accuracy)

print("-----------------------------------------------------------------------")

total_accuracy = (easy_correct + hard_correct) / (easy + hard)
print("Total accuracy: ", total_accuracy)





# asking the model about the informations of all the cards by id

id_answers = []
id_prompts = []

for i in range(10, 20):
    prompt = f"What are the details of the credit card with the ID {i}?"
    id_prompts.append(prompt)
    answer = get_answer(prompt)
    id_answers.append(answer)







# asking the model about the informations of all the cards by card number

number_answers = []
number_prompts = []

for i in range(7, 17):
    prompt = f"What are the details of the credit card with the number {i}?"
    number_prompts.append(prompt)
    answer = get_answer(prompt)
    number_answers.append(answer)



# aking multiple questions about the same card

exp_date_answers = []
exp_date_prompts = []

for i in range(10, 19):
    prompt = f"What is the expiration date of the credit card with the ID {i}?"
    exp_date_prompts.append(prompt)
    answer = get_answer(prompt)
    exp_date_answers.append(answer)



# asking about the last modification of the card

last_mod_answers = []
last_mod_prompts = []

for i in range(5, 13):
    prompt = f"What was the last modification made to the credit card with the ID {i}?"
    last_mod_prompts.append(prompt)
    answer = get_answer(prompt)
    last_mod_answers.append(answer)



# asking about the card type

type_answers = []
type_prompts = []

for i in range(10, 23):
    prompt = f"What is the type of the credit card with the ID {i}?"
    type_prompts.append(prompt)
    answer = get_answer(prompt)
    type_answers.append(answer)


# create a dataframe to store the results

id_test_results = pd.DataFrame()
id_test_results["Prompts"] = id_prompts
id_test_results["Answers"] = id_answers

number_test_results = pd.DataFrame()
number_test_results["Prompts"] = number_prompts
number_test_results["Answers"] = number_answers

exp_date_test_results = pd.DataFrame()
exp_date_test_results["Prompts"] = exp_date_prompts
exp_date_test_results["Answers"] = exp_date_answers

last_mod_test_results = pd.DataFrame()
last_mod_test_results["Prompts"] = last_mod_prompts
last_mod_test_results["Answers"] = last_mod_answers

type_test_results = pd.DataFrame()
type_test_results["Prompts"] = type_prompts
type_test_results["Answers"] = type_answers

id_test_results.to_csv("../Data/test/id_test_results.csv", index=False)
number_test_results.to_csv("../Data/test/number_test_results.csv", index=False)
exp_date_test_results.to_csv("../Data/test/exp_date_test_results.csv", index=False)
last_mod_test_results.to_csv("../Data/test/last_mod_test_results.csv", index=False)
type_test_results.to_csv("../Data/test/type_test_results.csv", index=False)





# Implement an interface to interact with the model

In [13]:
%pip install -q streamlit

Note: you may need to restart the kernel to use updated packages.


In [5]:
%%writefile RagApp.py

import streamlit as st

from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import DirectoryLoader , CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
import os

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
#from sentence_transformers import SentenceTransformerEmbeddings 

from langchain_community.llms import LlamaCpp
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)

from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

def init_page() -> None:
    st.set_page_config(
        page_title="IT COMP Q&A Chatbot Assistant", 
        layout="wide"
    )

    st.header("IT COMP Q&A Chatbot Assistant")

    st.sidebar.title("Options")

def select_llm() -> LlamaCpp:
    return LlamaCpp(
        model_path="..\models\mistral-7b-instruct-v0.2.Q4_K_M.gguf",
        temperature=0.75,
        max_tokens=2000,
        top_p=1,
        verbose=True,
    )

def load_embed_model():
    embed_model_Path = "..\models\sentence-transformers/all-MiniLM-L6-v2"

    embed_model = HuggingFaceEmbeddings(model_name=embed_model_Path)
    return embed_model 

def load_docs(directory):
    loader = CSVLoader(directory)
    documents = loader.load()
    return documents

def split_docs(documents, chunk_size=500, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    docs = text_splitter.split_documents(documents)
    return docs

def create_retriever(docs , embed_model):

    vector = FAISS.from_documents(docs, embed_model),
    print('*********************************************************************')
    print (vector)
    print('*********************************************************************')
    retriever = vector[0].as_retriever()
    
    return retriever


def load_chain(llm , retriever):
    
    prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

    <context>
    {context}
    </context>

    Question: {input}""")


    document_chain = create_stuff_documents_chain(llm, prompt)
    retrieval_chain = create_retrieval_chain(retriever, document_chain)

    return retrieval_chain


def init_chatbot() -> None:
    llm = select_llm()
    embed_model = load_embed_model()
    documents = load_docs("../Data/mini_data.csv")
    #docs = split_docs(documents)
    retriever = create_retriever(documents , embed_model)
    chain = load_chain(llm , retriever)
    return llm, retriever, chain


def init_messages() -> None:
    clear_button = st.sidebar.button("Clear conversation" , key="clear")
    if clear_button or "messages" not in st.session_state:
        st.session_state.messages = [
            SystemMessage(
                content="you are an AI assistant that can helps answering questions accurately provided by the context "
            ),
        ]

def get_answer(llm: LlamaCpp, retriever , chain , prompt: str) -> str:
    answer = chain.invoke({"input": prompt})
    answer = answer["answer"]
    return answer









def main() -> None:
    init_page()
    init_messages()
    llm, retriever, chain = init_chatbot()


    if st.chat_input("input your question here", key="input"):

        

        prompt = st.session_state.input

        with st.chat_message("user"):
            st.write(prompt)


        with st.spinner("Thinking..."):
            answer = get_answer(llm,retriever,chain, prompt)
            print(f"Answer: {answer}")

            message = st.chat_message("assistant")
            message.write(answer)

        #st.session_state.messages.append(SystemMessage(content=prompt))
        #st.session_state.messages.append(SystemMessage(content=answer))

   


if __name__ == "__main__":
    main()


Overwriting RagApp.py


In [7]:
!streamlit run RagApp.py